In [1]:
import warnings
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Title
import math

output_notebook()

Loading BokehJS ...

In [2]:
#ignore warning for now
warnings.filterwarnings('ignore')

data = pd.read_csv('Wholesale customers data-missing.csv')
original_data = pd.read_csv('Wholesale customers data.csv')

In [3]:
data.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656.0,7561.0,214,2674.0,1338.0
1,2,3,7057,9810.0,9568.0,1762,3293.0,1776.0
2,2,3,6353,8808.0,7684.0,2405,3516.0,7844.0
3,1,3,13265,1196.0,4221.0,6404,507.0,1788.0
4,2,3,22615,5410.0,7198.0,3915,1777.0,5185.0


In [4]:
data.isnull().sum()

Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
dtype: int64

In [5]:
data[data.Milk.isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
180,1,3,12356,NaN,8887.0,402,1382.0,2794.0


In [6]:
data[data.Detergents_Paper.isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
225,1,1,12680,3243.0,4157.0,660,NaN,786.0
370,2,3,39679,3944.0,4955.0,1364,NaN,2235.0


In [7]:
data[data.Grocery.isnull()]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
75,1,3,20398,1137.0,NaN,4407,3.0,975.0
172,1,3,955,5479.0,NaN,333,2840.0,707.0


In [8]:
data.shape


(440, 8)

In [9]:
#Handling missing value 1: Drop the data

# drop all rows with missing data
data_drop = data.dropna(how='any')


In [10]:
data_drop.shape

(434, 8)

In [11]:
#Drop a row if any of data in Milk or Delicassen are missing
data.dropna(subset=['Milk','Delicassen'], how='any').shape

(438, 8)

In [12]:
#Handle missing value 2: use a global constant
#Replace missing data with -1, inplace = True(make the change 
#in place in the dataframe)
data_with_const = data
data_with_const.fillna(value=-1, inplace=True)

In [13]:
data_with_const.isnull().sum()


Channel             0
Region              0
Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
dtype: int64

In [14]:
#Handle missing value 3: use the attribute mean
data_with_mean = data
data_with_mean.replace(-1, np.nan, inplace=True)
data_with_mean.iloc[225]
data_with_mean.isnull().sum()


Channel             0
Region              0
Fresh               0
Milk                1
Grocery             2
Frozen              0
Detergents_Paper    2
Delicassen          1
dtype: int64

In [15]:
#Find NaN value and replace it with the mean of the column

for label in data_with_mean.columns:
    if data_with_mean.loc[:][label].isnull().sum() > 0:
        mean_value = math.floor(data_with_mean.loc[:][label].mean())
        print(mean_value)
        data_with_mean.loc[:][label].fillna(value = mean_value, inplace=True)

data_with_mean.iloc[225][:]


5795.0
7972.0
2891.0
1525.0


Channel                 1.0
Region                  1.0
Fresh               12680.0
Milk                 3243.0
Grocery              4157.0
Frozen                660.0
Detergents_Paper     2891.0
Delicassen            786.0
Name: 225, dtype: float64

In [16]:
original_data.iloc[225][:]

Channel                 1
Region                  1
Fresh               12680
Milk                 3243
Grocery              4157
Frozen                660
Detergents_Paper      761
Delicassen            786
Name: 225, dtype: int64

In [17]:
#Method take in a data frame with its description of the dataset of type String
#Draw a scatter plot identifying relationship between milk and grocery of that datata
#Add interactions such as box zoom, box selection, pan, wheel zoom, reset
#Add hover tool to show specific data values

def visualize_data (data_frame, dataset_type):
    region_value = data_frame.loc[:, 'Region'] == 1
    

    full_title = 'ANNUAL SALE IN REGION 1 FOR MILK AND GROCERY ($)'
    
    p = figure(x_axis_label='groceries annual sale', y_axis_label= 'milk annual sale', 
               title = full_title, tools=['box_select,pan,wheel_zoom,box_zoom,reset'], 
               toolbar_location="right", toolbar_sticky=False)
    
    #Add subtitle at the bottom of the graph
    p.add_layout(Title(text=dataset_type, align="center"), "below")
    p.title.align="center"

    # Create a scatter plot 
    scatter_plot = p.circle( x=data_frame[region_value]['Grocery'], y=data_frame[region_value]['Milk'], 
                             size=10, color='green',  selection_color='red', nonselection_alpha=0.1, 
                             hover_fill_color='green', hover_alpha=0.5, hover_line_color='white')

    #Create a HoverTool to show value of annual sales in milk and grocery
    hover = HoverTool(renderers=[scatter_plot], tooltips=[('Milk ($)', '@y'), ('Groceries ($)', '@x') ])


    #Size of small ruler lines on axises 
    p.axis.minor_tick_in = 3
    p.axis.minor_tick_out = 6

    #Add the hover tool to the figure p
    p.add_tools(hover)
    show(p)
    
    return

In [18]:
#Visualize the complete dataset
visualize_data(original_data, 'complete dataset')

In [19]:
#Visualize the dataset using a global constant to handle missing values

visualize_data(data_with_const, 'use a global constant to handle missing values')

In [20]:
#Visualize the dataset using the attribute mean to handle missing values

visualize_data(data_with_mean, 'use the attribute mean to handle missing values')

In [21]:
#Visualize the dataset dropping rows to handle missing values

visualize_data(data_drop, 'drop rows to handle missing values')